In [1]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [99]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, clear_output

output1 = widgets.Output()
output2 = widgets.Output()
output3 = widgets.Output()
output4 = widgets.Output()
# create some x data
#x = np.linspace(0, 2 * np.pi, 100)

e1 = 0.25 #m
e2 = 0.05
k1 = 0.2 #W/(m.K)
k2 = 0.1
he2 = 40
Q=400 #W/m3
La = np.linspace(0,e1,10)
Lb = np.linspace(0,e2,10)
#Ltot = np.linspace(0,e1+e2,20)
Te = 300 #K
qpp = Q*e1

def T_B(Qa,ea,h,eb,kb):
    q = Qa*ea
    Tbr = Te + q/h
    Tbl = Tbr + q*eb/kb
    return Tbr, Tbl

def T_A(Qa,ea,ka, h, eb, kb):
    x = np.linspace(0,ea,10)
    x2 = np.linspace(0,eb,10)
    Ta = []
    L = []
    TbR, TbL = T_B(Qa,ea,h,eb,kb)
    for i in range(len(x)):
        Ta.append(-Qa*x[i]*x[i]/(2*ka)+((TbL-Te)/ea+Q*ea/(2*ka))*x[i]+Te)
        L.append(x[i])
    for i in range(1, len(x2)):
        Ta.append((TbR-TbL)*(x[len(x)-1]+x2[i])/x2[len(x2)-1]+(TbL*(x[len(x)-1]+x2[len(x2)-1])-TbR*x[len(x)-1])/x2[len(x2)-1])
        L.append(x[len(x)-1]+x2[i])
    return L, Ta

Ltot, Ttot = T_A(Q,e1,k1, he2, e2, k2)

# default line color
initial_color = '#FF00DD'

with output1:
    fig, ax = plt.subplots(constrained_layout=True, figsize=(8, 4))
     
# move the toolbar to the bottom
fig.canvas.toolbar_position = 'bottom'
ax.grid(True)
#line, = ax.plot(x_list, q_fin(x_list,k1,hi1,he1), initial_color, label='Q')
line, = ax.plot(Ltot, Ttot, color='b', label='T')
ax.set_xlim(0,0.425)
ax.set_ylim(299,400)
ax.set_xlabel('x')
ax.set_ylabel('T (K)')
ax.legend()

In [100]:
#output1

In [101]:
text_0 = widgets.HTML(value="<p>Consider the case of a composite wall - Wall A and Wall B with Wall A being heat generating. Assume there is not heat resistance between boundaries of Wall A and B. The left side of the Wall A is at ambient temperature of Tair = Tw = 300 K. Wall A has parameters La, Ka, with heat generation Q=400 W/m3. Wall B has parameters Lb, Kb, and h.</p>")

vbox_text = widgets.VBox([text_0])

In [102]:
# create some control elements
la_slider = widgets.FloatSlider(value=e1, min=0.15, max=0.35, step=0.05, description='La')
lb_slider = widgets.FloatSlider(value=e2, min=0.015, max=0.075, step=0.005, description='Lb')
ka_slider = widgets.FloatSlider(value=k1, min=0.05, max=2, step=0.05, description='Ka')
kb_slider = widgets.FloatSlider(value=k2, min=0.05, max=2, step=0.05, description='Kb')
heb_slider = widgets.FloatSlider(value=he2, min=30, max=50, step=1, description='h')
#Q_slider = widgets.FloatSlider(value=Q, min=390, max=500, step=10, description='Q')

# callback functions
def update1(change):
    """redraw line (update plot)"""
    xnew, ynew = T_A(Q,la_slider.value,ka_slider.value, heb_slider.value, lb_slider.value, kb_slider.value)
    line.set_xdata(xnew)
    line.set_ydata(ynew)
    fig.canvas.draw()

la_slider.observe(update1, 'value')
lb_slider.observe(update1, 'value')
ka_slider.observe(update1, 'value')
kb_slider.observe(update1, 'value')
heb_slider.observe(update1, 'value')
#Q_slider.observe(update1, 'value')


In [103]:
clear_output()
controls = widgets.VBox([vbox_text, la_slider, lb_slider, ka_slider, kb_slider, heb_slider])
page = widgets.HBox([controls, output1])
display(page)